# Case Study: Zeeguu/API - static analyze
- Backend of a web application that supports [free reading in foreign languages](https://zeeguu.org)
- Open source [repository on GH](https://github.com/zeeguu/API/)



# Basic Data Gathering

  - Basic used case, for creating a sequence diagram of the coding patch. 

  - file: `./zeeguu_core/model/user.py` <==>
  - module: `zeeguu_core.model.User`




In [8]:
# Installing Required Dependencies
import sys
sys.version
!{sys.executable} -m pip install gitpython
!{sys.executable} -m pip install pyvis

You should consider upgrading via the '/Users/nicklasjeppesen/opt/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nicklasjeppesen/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [9]:
# In Collab our notebook runs in a temporary mounted file system
# Let's print the name of the folder where our script runs
import os
cwd = os.getcwd()
print(cwd)


/Users/nicklasjeppesen/Desktop/software_individual_report/zeep


In [10]:
# Let's declare a var for the path where we're going to download a repository
# Warning: this must end in /
CODE_ROOT_FOLDER=cwd+"/content/zeeguu_api/"


In [11]:
from git import Repo
# GitPython is a library that allows us to work easily with git from Python
# https://gitpython.readthedocs.io/en/stable/tutorial.html


# If the file exists, it means we've already downloaded
#if not os.path.exists(CODE_ROOT_FOLDER):
if not os.path.exists(CODE_ROOT_FOLDER):
  Repo.clone_from("https://github.com/zeeguu/api", CODE_ROOT_FOLDER)



# how it work: 

## functions in code library
* module_name_from_file_path: where we get a current module name from a file path. 
* file_path_from_module_name: return file path from a module name as string. 
* imports_from_file(file): return array of all module import of a file, where module is from zeguu, otherwise empty. 
* find_python_functions(file): # Find all functions, defined with def first, from a text file. 
* extract_function_body(func_name, func_string): Return a function body from string, where the string can be a string of a class and not. 
* getCalledfunction(source_code): Return a list of called functions from a function as string    
* getFileContentFromPath(file_path): get content of file by a file path. 

Process: 

1. Get the function body of the examing function by code path to the file and def name. 
    * function body: getFileContentFromPath, extract_function_body(func_name, filecontent)
2. Get a list of all calling functions in that function: 
    * list of calling functions: getCalledfunction(source_code):
3. foreach called functions, 
    if they are in one of zeeguu module, 
        get filepath by module name, and call recusive, just to print, calling hiracy. 
        Next, return a calling node, of functions. 


master function: 
    def masterfunction(file_path, function_name): 
        Execute. 




# Code library: 



In [18]:
# naïve way of extracting imports using regular expressions
import re
import builtins

def getFileContentFromPath(file_path): 
    with open(file_path, 'r') as file:
        file_content = file.read()
    return file_content

# extracting a module name from a file name
# e.g. ../core/model/user.py -> zeeguu.core.model.user
def module_name_from_file_path(full_path):   
    file_name = full_path[len(CODE_ROOT_FOLDER):]
    file_name = file_name.replace("/__init__.py","")
    file_name = file_name.replace("/",".")
    file_name = file_name.replace(".py","")
    return file_name


# extracting a module name from a file name
# e.g. 'zeeguu.core.model.user' --> cwd+/content/zeeguu-api/zeeguu/core/model/user.py 
# There can be two issues: 
# 1: if the module also include a functionCall, then we shall remove it. 
# 2: if the class named, is not equal the fileName, then we shall check the init file, the find the file name. 

def getCorrectFileNameFromInitFile(init_path, className): 
    lines = [line for line in open(init_path)]
    for line in lines:
        imp = className in line

        if imp: 
            lineSplit = line.split(" ")
            newLine = lineSplit[1]
            newLine = newLine.replace(".", "")
            return newLine
            
# extracting a module name from a file name
# e.g. 'zeeguu.core.model.user' --> cwd+/content/zeeguu-api/zeeguu/core/model/user.py 
# There can be two issues: 
# 1: if the module also include a functionCall, then we shall remove it. 
# 2: if the class named, is not equal the fileName, then we shall check the init file, the find the file name. 
# 3: Something we import a function, where we define the fileName in the from statement, and then the function, in the import. 
def file_path_from_module_name(module_name):
    file_name = module_name.replace(".","/")
    firstTry = CODE_ROOT_FOLDER+file_name+".py"
   
    if(os.path.exists(firstTry)): # check if modulename, is correct file path.  
        return firstTry
   
    # problem 1: try to remove function name, from file/class name, so we only have, module and class/file name. 
    firstTry = CODE_ROOT_FOLDER+file_name # removing the .py, for easier manipulate the path later. 
    recon = firstTry.rsplit("/", 1)
    simiFirstTry = recon[0]+".py"
    if(os.path.exists(recon[0]+".py")): 
        return recon[0]+".py"
    #if(not os.path.isdir(recon[0])): 
       
     #   recon = recon[0].rsplit("/", 1)
        #if(not os.path.isdir(recon[0])): 
         #   return None # I dont know what to do longer, so give up. 
    
    # problem 2: hope, module is equal fileName, but not sure. 
    retryTwo = recon[0]+"/"+recon[1]+".py"
    if(os.path.exists(retryTwo)): # if True, then the className/filename/Module name is correct. 
       return retryTwo

    init_path = recon[0]+"/__init__.py"
    correctFileName = getCorrectFileNameFromInitFile(init_path, recon[1])
    
    if not correctFileName: 
        return None
    retryThree = recon[0]+"/"+correctFileName+".py"
    if(os.path.exists(retryThree)): 
        return retryThree
    
    #Problem 3: from is a dir, and import is the function name. 
    
    return None
 

 

# helper function to get a file path w/o having to always provide the /content/zeeguu-api/ prefix
def file_path(file_name):
    return CODE_ROOT_FOLDER+file_name

#assert 'zeeguu.core.model.user' == module_name_from_file_path(file_path('zeeguu/core/model/user.py'))

# we assume that imports are always at the
# MODIFIED, 
#   SO IT RETURN IMPORT FROM OTHER PARTS OF THE ZEEGUU CODE BASED. 
#   Also, it return an array, instead of a single 
def import_from_line(line):
    # regex patterns used
    #   ^  - beginning of line
    #   \S - anything that is not space
    #   +  - at least one occurrence of previous
    #  ( ) - capture group (read more at: https://pynative.com/python-regex-capturing-groups/)
    finalArray = []
    try:
      y = re.search("^from (\S+)", line)
      if y:
        for item in getImportedModules(line): 
            
            if(item.startswith("zeeguu")):
                finalArray.append(item)
      else:
        y = re.search("^import (\S+)", line)
        if y.group(1).startswith("zeeguu"):
            finalArray.append(y.group(1))
      #if(module.startswith("zeeguu")): 
       # return module
     # else:
      #  return None
      #return y.group(1)
    except:
      return finalArray
    return finalArray

def getImportedModules(text2):
    y = re.search("^from (\S+)", text2)
    if y: 
       
        basisModule = y.group(1)
        finalModuleList = []
        pattern = r"from\s+[\w.]+\s+import\s+([\w\s,]+)"
        match = re.search(pattern, text2)
        if match:
            imported_items_str = match.group(1)
            imported_items = [item.strip() for item in imported_items_str.split(",")]
            
            for item in imported_items: 
                finalModuleList.append(basisModule+"."+item)
            return finalModuleList
        else:
            return []
        


# extracts all the imported modules from a file
# returns a module of the form zeeguu_core.model.bookmark, e.g.
def imports_from_file(file):
    all_imports = []
    lines = [line for line in open(file)]

    for line in lines:
        imp = import_from_line(line)

        if imp:
            #all_imports.append(imp)
            for item in imp: 
                all_imports.append(item)

    return all_imports

#imports_from_file(file_path('/zeeguu/core/model/user.py'))
#imports_from_file(file_path('/zeeguu/api/endpoints/activity_tracking.py'))


# Find all functions, defined with def first, from a text file. 
def find_python_functions(file):
    pattern = r"def\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\([^()]*\)\s*:"
    matches = re.findall(pattern, file)
    return matches


# Return a function body from string, where the string can be a string of a class and not. 
def extract_function_body(func_name, func_string):
    #pattern = r'def\s+' + re.escape(func_name) + r'\s*\(.*?\)\s*:\s*(.*?)\bdef\s'
    #pattern = rf"def\s+{func_name}\s*\(\s*\):(\s*\n\s*\"\"\".*?\"\"\")?\s*(.*?)\bdef\s"
    pattern = "(def*\W("+func_name+"+)([\w\W]*?))(?=(def)|$)"

    match = re.search(pattern, func_string, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return None

# Return a list of called functions from a function as string    
def getCalledfunction(source_code):
   
    builtinsFunctions = dir(__builtins__)
    called_functions = []

    # Define the regex pattern to match function calls
    function_call_pattern = r'(\w+)\('
    # Find all matches of function calls in the source code
    matches = re.findall(function_call_pattern, source_code)

    # Filter out function names that are not defined in the global scope
    for match in matches:
        if match not in builtinsFunctions:# and callable(globals()[match]):
            called_functions.append(match)
    return called_functions
 

# return a new list, of function there exists in the modules list. 
def GetAllRealFilePath(list_of_functions: list, modules: list, chooseModule = False): 
    functions_that_exists = []
    for func in list_of_functions: 
        for module in modules: 
            try: 
                mini_file_path = file_path_from_module_name(module) # return a specific file path, for a given module. 
                file_content = getFileContentFromPath(mini_file_path)
               
                functions_in_folder = find_python_functions(file_content)
                if func in functions_in_folder: 
                    if(chooseModule): 
                        newValue = module+";"+func
                        functions_that_exists.append(newValue)
                        break
                    else: 
                        newValue = mini_file_path+";"+func
                        functions_that_exists.append(newValue)
                        break
            except Exception as e: 
                #print("error at module: " + module)
                #sprint("error at function: " + func)
                e
    return functions_that_exists

def getTheRealCallingFunction(spliter): 
    realfunctions = []
    for realfunc in spliter:
        dd =  realfunc.split(";")
        realfunctions.append(dd[1])
    return realfunctions

'''
# return a list of string in this format <modulename>;<functionname>
by providing a function name and path to the file, that have the function in it. 

'''
def getfunctionsModuleUsedInFunction(func_name, endpoint):
    codeContent = getFileContentFromPath(endpoint) 
    funcBody = extract_function_body(func_name, codeContent)

    CalingFunctions = getCalledfunction(funcBody) # return a list of calling functions ex. ['upload_user_activity_data', 'create_from_post_data', 'get', 'distill_article_interactions', 'get', 'notify_audio_experiment']
    
    if func_name in CalingFunctions: CalingFunctions.remove(func_name)
    
    ImportedModules = imports_from_file(endpoint) # return list of modules (might be functionNames in them) ex. ['zeeguu.api.utils.route_wrappers.cross_domain', 'zeeguu.api.utils.route_wrappers.with_session', 'zeeguu.core.model.UserActivityData']
    realFunctionsToPrint = GetAllRealFilePath(CalingFunctions, ImportedModules, True) # return a list of <filepath>;<function_namer>, that is used in parameter function body. 
    realFunctions = GetAllRealFilePath(CalingFunctions, ImportedModules) # return a list of <filepath>;<function_namer>, that is used in parameter function body. 

    '''   
    for realfunc in realFunctions: 
        file_path, _func = realfunc.split(";")
        masterFunction(_func, file_path)
    '''

    return realFunctionsToPrint


# create the needed module Func string, based on the method GetAllRealFilePath, where excepted is, Module;func structure. 
def createMermaidString(moduleFunc: list): 
    firstModule, firstfunc = moduleFunc[0].split(";")
    finalString = "sequenceDiagram\n"
    
    # add all module to the file. 
    for _module in moduleFunc: 
        module, func = _module.split(";")
        finalString = finalString + "participant "+module+"\n"
    
    moduleFunc.pop(0)
    for _module in moduleFunc: 
        module, func = _module.split(";")
        finalString = finalString + firstModule+"->>+"+module+": "+ func +"\n"
        finalString = finalString + module+"-->>-"+firstModule+": return"  +"\n"
    return finalString



def generate_sequence_diagram(mermaid_code, output_file):
    import subprocess
    try:
        # Write Mermaid code to a temporary file
        with open("temp.mmd", "w") as f:
            f.write(mermaid_code)
        
        # Call Mermaid CLI to generate the diagram
        subprocess.run(["mmdc", "-i", "temp.mmd", "-o", output_file], check=True)
        
        print("Sequence diagram generated successfully!")
    except subprocess.CalledProcessError as e:
        print("Error generating sequence diagram:", e)
    finally:
        # Clean up temporary file
        subprocess.run(["rm", "temp.mmd"])


# Master function, to create a sequence diagram, depends on func_name and endpoint:
def analyze(func_name, endpoint): 
    moduleFunc = getfunctionsModuleUsedInFunction(func_name, endpoint)
    starModuleName = module_name_from_file_path(endpoint)
    startModule = starModuleName+";"+func_name
    moduleFunc.insert(0, startModule)

    finalMerMaidString = createMermaidString(moduleFunc)
    output_file = func_name+".png"

    generate_sequence_diagram(finalMerMaidString, output_file)







# Run Code

In [23]:
ENDPOINT_ONE = CODE_ROOT_FOLDER+"zeeguu/api/endpoints/activity_tracking.py"
ENDPOINT_TWO = CODE_ROOT_FOLDER+"zeeguu/api/endpoints/user_articles.py"
os.path.exists(ENDPOINT_ONE)
os.path.exists(ENDPOINT_TWO)

True

In [25]:
# used of  code library: 

func_name1 = "upload_user_activity_data"
func_name2 = "user_articles_recommended"


analyze(func_name1, ENDPOINT_ONE)
analyze(func_name2, ENDPOINT_TWO)



Sequence diagram generated successfully!
Sequence diagram generated successfully!
